# Information
This notebook is about to give you some **examples** about how to use the notebooks and associated functionalities in the **beak** package.

# Examples

## 1. Create a list of raster files based on a pre-defined model definition.
Case:
- You have a list of raster files in a folder or some subfolders.
- You have a model definition for a specific model, stored in the respective model module.
- You need a list of files corresponding to the evidence layers defined in the model definition.

Load model definition

In [1]:
from beak.models import mvt_nat

MODEL = "MVT_PREFERRED"
model = mvt_nat.models[MODEL]

print("Model definition:")
print("-----------------")
for layer, value in model.items():
  if value is True: 
    print(layer)
  

Model definition:
-----------------
Geology_Lithology_Majority
Geology_Lithology_Minority
Geology_Period_Maximum_Majority
Geology_Period_Minimum_Majority
Geology_Dictionary_Calcareous
Geology_Dictionary_Carbonaceous
Geology_Dictionary_FineClastic
Geology_Dictionary_Felsic
Geology_Dictionary_Intermediate
Geology_Dictionary_UltramaficMafic
Geology_Dictionary_Anatectic
Geology_Dictionary_Gneissose
Geology_Dictionary_Schistose
Terrane_Proximity
Geology_PassiveMargin_Proximity
Geology_BlackShale_Proximity
Geology_Fault_Proximity
Geology_Paleolatitude_Period_Maximum
Geology_Paleolatitude_Period_Minimum
Gravity_GOCE_ShapeIndex
Gravity_Bouguer
Gravity_Bouguer_HGM
Gravity_Bouguer_UpCont30km_HGM
Gravity_Bouguer_HGM_Worms_Proximity
Gravity_Bouguer_UpCont30km_HGM_Worms_Proximity
Magnetic_HGM
Magnetic_LongWavelength_HGM
Magnetic_HGM_Worms_Proximity
Magnetic_LongWavelength_HGM_Worms_Proximity
Seismic_LAB_Hoggard
Seismic_Moho


Load file paths: You can use the practical handling from the package, if data are stored in the beak data folder.<br>
Also, you can provide multiple folders to load from

In [4]:
from importlib_resources import files

BASE_PATH = files("beak.data") / "LAWLEY22-EXPORT" / "EPSG_4326_RES_0_05" / "COMPLETE_DATASET"

PATH_NUMERICAL = BASE_PATH / "NUMERICAL_MINMAX"
PATH_CATEGORICAL = BASE_PATH / "CATEGORICAL"


In [7]:
from beak.utilities.io import load_model

layers, matches, counts = load_model(model=model, 
                                     folders=[PATH_NUMERICAL, PATH_CATEGORICAL], 
                                     file_extensions=[".tif", ".tiff"], 
                                     verbose=0)

Loading model definition...
Selected 31 evidence layers.

Searching for files and folders in provided paths...
Found 2 folders, 43 subfolders and 812 files.

Searching for corresponding files...
Found 'Geology_Lithology_Majority' in 31 file(s).
Found 'Geology_Lithology_Minority' in 31 file(s).
Found 'Geology_Period_Maximum_Majority' in 20 file(s).
Found 'Geology_Period_Minimum_Majority' in 20 file(s).
Found 'Geology_Dictionary_Calcareous' in 1 file(s).
Found 'Geology_Dictionary_Carbonaceous' in 1 file(s).
Found 'Geology_Dictionary_FineClastic' in 1 file(s).
Found 'Geology_Dictionary_Felsic' in 1 file(s).
Found 'Geology_Dictionary_Intermediate' in 1 file(s).
Found 'Geology_Dictionary_UltramaficMafic' in 1 file(s).
Found 'Geology_Dictionary_Anatectic' in 1 file(s).
Found 'Geology_Dictionary_Gneissose' in 1 file(s).
Found 'Geology_Dictionary_Schistose' in 1 file(s).
Found 'Terrane_Proximity' in 1 file(s).
Found 'Geology_PassiveMargin_Proximity' in 1 file(s).
Found 'Geology_BlackShale_Prox